# Computational Finance Assignment1

In [ ]:
import math
import csv
import time
import sympy as sp
import numpy as np
import scipy.stats as si
import matplotlib.pyplot as plt

## Part I Theory: derivatives and no-arbitrage

### 1.Prove:

`A = C(1 + r/n)^(nt) = C * e^(rt)`

The standard compound interest formula is: 

`A = C(1 + r/n)^(nt)`, where:

- `A` is the future value,
- `C` is the principal (initial investment),
- `r` is the annual interest rate,
- `n` is the number of compound interest per year,
- `t` is time in years.

As we let `n` go to infinity, we can use the property of the base `e` of the natural logarithm, `(1 + 1/n)^n` tends to `e` as `n` goes to infinity. We set `n = 1,10,100,1000,10000` to compare these two results


In [ ]:
C = 1 
r = 0.1
t = 1

A_true = C * math.exp(r * t) 
ns = [1,10,100,1000,10000]
for n in ns:
    A_n= C * pow(1 + r/n, n * t)
    absolute_error = abs(A_true - A_n)/A_true
    print(f"Continuous compound interest : {A_true} ", f"Standard compound interest (with n = {n}): {A_n}")
    print(f"Absolute Error:{absolute_error}")
    print("\n")


### 2.Calculate the fair-value of a coupon bond.

In [ ]:


# Calculate the fair value of a bond
# Given parameters
face_value = 50000  # Face value of the bond
coupon = 300  # Coupon payment per quarter
maturity = 2  # Maturity period in years
risk_free_rate = 0.015  # Risk-free interest rate
quarters = maturity * 4  # Total number of quarters for payments

# Calculate the present value of each quarter
present_value_coupons = sum(coupon * sp.exp(-risk_free_rate * t/4) for t in range(1, quarters + 1))

# Present value of the final principal payment
present_value_face_value = face_value * sp.exp(-risk_free_rate * maturity)

# The fair value of the bond is the sum of present values of all cash flows
fair_value_bond = present_value_coupons + present_value_face_value

print(f"Fair-value of this coupon bond : {fair_value_bond.evalf()}")  # Evaluate the numerical result of the expression




### 3.Calculate forward price of a contract at time zero

In financial markets, the fair forward price of a contract at time zero is often calculated using the no-arbitrage principle. This principle dictates that the price of derivative securities must be set in a way that does not allow for arbitrage opportunities through risk-free trades with guaranteed profits. The fair forward price, denoted by `F_0`, is expressed as `S_0 * e^(rT)`, which indicates that the forward price is equivalent to the current price of the underlying asset, `S_0`, compounded at the risk-free interest rate r over the time to maturity T.

Thus, if the actual forward price in the market deviates from `S_0 * e^(rT)`, arbitrageurs can exploit this difference to secure risk-free profits. 

If the market forward price is higher than `S_0 * e^(rT)`, it is profitable to sell the forward contract and invest the current underlying asset price `S_0` at the risk-free rate r. 

Conversely, if the market forward price is lower than `S_0 * e^(rT)`, one can achieve a risk-free profit by entering into a forward contract to buy the asset and simultaneously borrowing the amount `S_0` to purchase the underlying asset. This process will continue until the market forward price adjusts to match the no-arbitrage price of `S_0 * e^(rT)`.

### 4. Draw the pay-oﬀ diagrams for two portfolios

1. A call option and an investment of `Ke^{-rT}` in the money-market; 
2. A put option and one share of the stock `S`.


In [ ]:

# Define the parameters
K = 50  # Strike price
r = 0.05  # Risk-free rate
T = 1  # Time to maturity
S0 = 50  # Current stock price

# Define the range of stock prices at maturity
ST = np.linspace(0, 100, 500)

# Calculate the payoff for a call option at maturity
call_payoff = np.maximum(ST - K, 0)
# Calculate the present value of the investment in the money market
money_market_investment = K * np.exp(-r * T)
# Total payoff for the first portfolio (call option + money market investment)
portfolio1_payoff = call_payoff + money_market_investment

# Calculate the payoff for a put option at maturity
put_payoff = np.maximum(K - ST, 0)
# Total payoff for the second portfolio (put option + stock)
portfolio2_payoff = put_payoff + ST

# Plot the pay-off diagrams for both portfolios
plt.figure(figsize=(14, 7))

# Portfolio 1: Call option and money market investment
plt.subplot(1, 2, 1)
plt.plot(ST, portfolio1_payoff, label='Call Option + Money Market Investment')
plt.title('Portfolio 1 Payoff at Maturity')
plt.xlabel('Stock Price at Maturity (ST)')
plt.ylabel('Profit')
plt.legend()
plt.grid(True)

# Portfolio 2: Put option and stock
plt.subplot(1, 2, 2)
plt.plot(ST, portfolio2_payoff, label='Put Option + Stock')
plt.title('Portfolio 2 Payoff at Maturity')
plt.xlabel('Stock Price at Maturity (ST)')
plt.ylabel('Profit')
plt.legend()
plt.grid(True)

# Show the plots
plt.tight_layout()
plt.show()


Portfolio 1 Payoff at Maturity (Call Option + Money Market Investment): This graph shows that if the stock price at maturity (ST) is below the strike price (K), the payoff is constant because the call option is not exercised and the investor only gets the money market investment value. As the stock price exceeds the strike price, the payoff increases linearly with ST since the call option is exercised.

Portfolio 2 Payoff at Maturity (Put Option + Stock): This graph illustrates that if the stock price at maturity is above the strike price, the payoff is linear and increases with ST because the put option is not exercised, and the investor owns the stock. When the stock price is below the strike price, the payoff is constant because the put option is exercised, ensuring the investor sells the stock at the strike price K.

The linear payoff beyond the strike price reflects the characteristics of owning the underlying asset (stock) with the protection of an option. The flat part of the payoff represents the protective feature of the option where the loss is limited to the premium paid for the option. ​

### 5. Prove put-call parity：`C_t + Ke^(-r(T-t)) = P_t + S_0`
#### Portfolio A:
- Consists of one European call option with the value `C_t` and an investment of `Ke^(-r(T-t))` in the risk-free asset. The payoff at maturity is the greater of `S_T - K` or 0, plus `K`, which simplifies to `max(S_T, K)`.

#### Portfolio B:
- Consists of one European put option with the value `P_t` and one share of the stock `S_0`. The payoff at maturity is the greater of `K - S_T` or 0, plus `S_T`, which simplifies to `max(S_T, K)`.

Given that the payoff of both portfolios is the same at maturity, and no arbitrage opportunities exist, their value should be equal at the present time. This leads to the equation:

`C_t + Ke^(-r(T-t)) = P_t + S_0`


## Part II: Binomial tree: option valuation

#### 1. Build the binomial rree and test

In [ ]:
def build_tree(S, vol, T, N):
    '''
    Build a binomial tree for option pricing.
    Paramers:
        S: Initial stock price
        vol: Volatility
        T: Time to expiration
        N: Number of steps
    Output:
        A 2D array representing the binomial tree
    '''
    dt = T / N  # Time step
    u = np.exp(vol * np.sqrt(dt))  # Up factor
    d = 1 / u  # Down factor
    matrix = np.zeros((N + 1, N + 1))  # Create a 2D array for tree to store the stock prices
    
    for i in range(N + 1): 
        for j in range(i + 1):  
            matrix[i, j] = S * (u ** j) * (d ** (i - j))
            
    return matrix

def value_option_matrix(tree, T, r, K, vol):
    '''
    Calculate the option value using a binomial tree.
    Parameters:
        tree: Binomial tree
        T: Time to expiration
        r: Risk-free interest rate
        K: Strike price
        vol: Volatility
    Output:
        Option value at the root of the tree
    '''
    N = tree.shape[0] - 1  # Number of steps
    dt = T / N 
    u = np.exp(vol * np.sqrt(dt)) 
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)  # Risk-neutral probability

    for c in range(N + 1):
        tree[N, c] = max(0, tree[N, c] - K) # Option value at expiration
    
    for i in range(N - 1, -1, -1): 
        for j in range(i + 1):
            # Calculate the expected value of the option
            tree[i, j] = np.exp(-r * dt) * (p * tree[i + 1, j + 1] + (1 - p) * tree[i + 1, j])
            
    return tree[0, 0]

In [ ]:
sigma = 0.2  # Volatility
S = 100  # Current stock price
T = 1.0  # Time to maturity
N = 50  # Number of steps
K = 99  # Strike price
r = 0.06  # Risk-free interest rate

tree = buildTree(S, sigma, T, N)
option_price = valueOptionMatrix(tree, T, r, K, sigma)
option_price

#### 2.  Compare with the Black-Scholes fomula on value of option
First we compare the difference between the Black-Scholes fomula and the binomial tree's computed result.

In [ ]:
def black_scholes_call(S, K, T, r, sigma):
    """
    Computes the Black-Scholes formula for a European call option.
    
    Parameters:
        S : float
            Current stock price
        K : float
            Strike price
        T : float
            Time to expiration
        r : float
            Risk-free interest rate
        sigma : float
            Volatility of the underlying stock
    
    Returns:
        call_price : float
            Price of the European call option
    """
    # Calculate d1 and d2 parameters
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    # Calculate call price using cumulative distribution functions
    call_price = (S * si.norm.cdf(d1) - K * np.exp(-r * T) * si.norm.cdf(d2)) 
    
    return call_price

def compare_volatilities(S, K, T, r, vol_range, N):
    '''
    Compare option prices for different volatilities.
    Parameters:
        S : float
            Current stock price
        K : float
            Strike price
        T : float
            Time to expiration
        r : float
            Risk-free interest rate
        vol_range : array
            Array of volatilities
    Returns:
        bs_prices : array
            Array of option prices for different volatilities under Black-Scholes model and binomial tree model
    '''
    bs_prices = []
    bt_prices = []
    for sigma in vol_range:
        bs_price = black_scholes_call(S, K, T, r, sigma)
        bs_prices.append(bs_price)
        tree = buildTree(S, sigma, T, N)
        option_price = valueOptionMatrix(tree, T, r, K, sigma)
        bt_prices.append(option_price)
    return bs_prices, bt_prices


In [ ]:
sigma = 0.2  
S = 100 
T = 1.0 
N = 50  
K = 99 
r = 0.06 
vol_range = np.linspace(0.1, 0.99, 100)
N_range = [20, 50, 200]
# Plot the option prices for different volatilities
plt.figure(figsize=(20, 5))
for i, N in enumerate(N_range):
    bs_prices, binomial_prices = compare_volatilities(S, K, T, r, vol_range, N)
    plt.subplot(1, 3, i+1)
    plt.plot(vol_range, bs_prices, label='Black-Scholes Prices')
    plt.plot(vol_range, binomial_prices, label='Binomial Tree Prices', linestyle='--')
    plt.title(f'European Call Option Price for N = {N}', fontsize=16)
    plt.xlabel('Volatility', fontsize=16)
    plt.ylabel('Call Option Price', fontsize=16)
    plt.legend(fontsize=14)
    plt.grid(True)
plt.show()



In [ ]:
sigma = 0.2  
S = 100 
T = 1.0 
N = 50  
K = 99 
r = 0.06 
vol_range = np.linspace(0.1, 0.99, 100)
N_range = [20, 50, 200]
# Plotting both Black-Scholes and Binomial tree prices difference for comparison
plt.figure(figsize=(20, 5))
for i, N in enumerate(N_range):
    bs_prices, binomial_prices = compare_volatilities(S, K, T, r, vol_range, N)
    plt.subplot(1, 3, i+1)
    plt.plot(vol_range, np.array(bs_prices) - np.array(binomial_prices), label='Difference in Option Prices')
    plt.title(f'European Call Option Price for N = {N}', fontsize = 16)
    plt.xlabel('Volatility', fontsize=16)
    plt.ylabel('Call Option Price', fontsize=16)
    plt.ylim(-0.05, 0.4)
    plt.legend(fontsize = 12)
    plt.grid(True)
plt.show()

In [ ]:
sigma = 0.2  
S = 100 
T = 1.0 
N = 50  
K = 99 
r = 0.06 
vol_range = np.linspace(0.1, 0.99, 100)
N_range = [20, 50, 200]
plt.figure(figsize=(20, 5))
for i, N in enumerate(N_range):
    bs_prices, binomial_prices = compare_volatilities(S, K, T, r, vol_range, N)
    plt.subplot(1, 3, i+1)
    plt.plot(vol_range, (np.array(bs_prices) - np.array(binomial_prices))/np.array(bs_prices), label='Relative Difference in Option Prices', linestyle='--')   
    plt.title(f'Option Price Relative Difference for N = {N}', fontsize = 16)
    plt.xlabel('Volatility', fontsize=16)
    plt.ylabel('Call Option Price', fontsize=16)
    plt.ylim(-0.001, 0.01)
    plt.legend(fontsize = 12)
    plt.grid(True)
plt.show()


In [ ]:
sigma = 0.2  
S = 100 
T = 1.0 
N = 50  
K = 99 
r = 0.06 
vol_range = np.linspace(0.1, 0.99, 100)
N_range = [20, 50, 200]

# Plotting the time taken for option pricing for different volatilities
plt.figure(figsize=(20, 5))
for i, N in enumerate(N_range):

    bs_times = []
    bt_times = []
    for vol in vol_range:
        start_bs = time.time()
        bs_prices = black_scholes_call(S, K, T, r, vol)
        end_bs = time.time()
        bs_time = end_bs - start_bs
        bs_times.append(bs_time)
        start_bt = time.time()
        bt_prices = value_option_matrix(build_tree(S, vol, T, N), T, r, K, vol)
        end_bt = time.time()
        bt_time = end_bt - start_bt
        bt_times.append(bt_time)
    plt.subplot(1, 3, i+1)
    plt.plot(vol_range, bs_times, label='Black-Scholes')
    plt.plot(vol_range, bt_times, label='Binomial Tree', linestyle='--')
    plt.title(f'Time taken for Option Pricing for N = {N}', fontsize = 16)
    plt.xlabel('Volatility', fontsize=16)
    plt.ylabel('Time (seconds)', fontsize=16)
    plt.ylim(0, 0.06)
    plt.legend(fontsize = 12)
    plt.grid(True)
plt.show()

#### 3. Convergence

In [ ]:
sigma = 0.2  
S = 100 
T = 1.0 
N = 50  
K = 99 
r = 0.06 
vol_range = np.linspace(0.1, 0.99, 100)
N_range = [20, 50, 200]
N_values = [10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200,300,500]
volatility_values = [0.1,0.2,0.5,0.9]

# Plot the option value convergence when increasing the number of steps in the binomial tree
plt.figure(figsize=(25, 5))
for i, vol in enumerate(volatility_values):
    option_values = []
    for N in N_values:
        tree = build_tree(S, vol, T, N)
        option_price = value_option_matrix(tree, T, r, K, vol)
        option_values.append(option_price)
    plt.subplot(1, 4, i+1)
    plt.plot(N_values, option_values, label='Option Value')
    plt.title(f'Option Value Convergence for Volatility = {vol}', fontsize = 15)
    plt.xlabel('Number of Steps', fontsize=16)
    plt.ylabel('Option Value', fontsize=16)
    plt.legend(fontsize = 12)
    plt.grid(True)

In [ ]:
sigma = 0.2  
S = 100 
T = 1.0 
N = 50  
K = 99 
r = 0.06 
vol_range = np.linspace(0.1, 0.99, 100)
N_range = [20, 50, 200]
N_values = [10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200,300,400,500]
volatility_values = [0.1,0.2,0.5,0.9]

# Plot the average compute time when increasing the number of steps in the binomial tree
plt.figure(figsize=(25, 5))
for i, vol in enumerate(volatility_values):
    compute_times = []
    for N in N_values:
        compute_times_1 = []
        for j in range(10):
            start = time.time()
            tree = build_tree(S, vol, T, N)
            option_price = value_option_matrix(tree, T, r, K, vol)
            end = time.time()
            compute_time = end - start
            compute_times_1.append(compute_time)
        compute_times.append(np.mean(compute_times_1))
    plt.subplot(1, 4, i+1)
    plt.plot(N_values, compute_times, label='Compute Time')
    plt.title(f'Compute Time for Volatility = {vol}', fontsize = 15)
    plt.xlabel('Number of Steps', fontsize=16)
    plt.ylabel('Time (seconds)', fontsize=16)
    plt.ylim(0,0.2)
    plt.legend(fontsize = 12)
    plt.grid(True)

In [ ]:
sigma = 0.2  
S = 100 
T = 1.0 
N = 500 
K = 99 
r = 0.06 
vol_range = np.linspace(0.1, 0.99, 40)

# Plot the average compute time with different volatilities
plt.figure(figsize=(7, 6))
compute_times = []
for vol in vol_range:
    compute_times_1 = []
    for j in range(10):
        start = time.time()
        tree = build_tree(S, vol, T, N)
        option_price = value_option_matrix(tree, T, r, K, vol)
        end = time.time()
        compute_time = end - start
        compute_times_1.append(compute_time)
    compute_times.append(np.mean(compute_times_1))
plt.plot(vol_range, compute_times, label='Compute Time')
plt.title(f'Compute Time for N = {N}', fontsize = 16)
plt.xlabel('Volatility', fontsize=16)
plt.ylabel('Time (seconds)', fontsize=16)

plt.legend(fontsize = 12)
plt.grid(True)
plt.show()


In [ ]:
# write the compute_tines above into a csv file

with open('\data\compute_times.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Volatility", "Compute Time"])
    for i in range(len(vol_range)):
        writer.writerow([vol_range[i], compute_times[i]])
        
# Read the csv file into a pandas dataframe
import pandas as pd



#### 4. Compare \delta

In [ ]:
def binomial_tree_delta(S, K, T, r, sigma, N):
    '''
    Compute the delta of a European call option using a binomial tree.
    Parameters:
        S : float
            Current stock price
        K : float
            Strike price
        T : float
            Time to expiration
        r : float
            Risk-free interest rate
        sigma : float
            Volatility
        N : int
            Number of steps
    Returns:
        Delta : float
            Delta of the option
    '''
    dt = T / N  # Time step
    u = np.exp(sigma * np.sqrt(dt))  # Up factor
    d = 1 / u  # Down factor
    p = (np.exp(r * dt) - d) / (u - d)  # Risk-neutral probability
    tree = build_tree(S, sigma, T, N)
    option_price = value_option_matrix(tree, T, r, K, sigma)
    tree_up = build_tree(S * u, sigma, T, N)
    option_price_up = value_option_matrix(tree_up, T, r, K, sigma)
    tree_down = build_tree(S * d, sigma, T, N)
    option_price_down = value_option_matrix(tree_down, T, r, K, sigma)
    Delta = (option_price_up - option_price_down) / (S * (u - d))
    return Delta

def black_scholes_delta(S, K, T, r, sigma):
    '''
    Compute the delta of a European call option using the Black-Scholes formula.
    Parameters:
        S : float
            Current stock price
        K : float
            Strike price
        T : float
            Time to expiration
        r : float
            Risk-free interest rate
        sigma : float
            Volatility
    Returns:
        Delta : float
            Delta of the option'''
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    Delta = si.norm.cdf(d1)
    return Delta

In [ ]:
sigma = 0.2  
S = 100 
T = 1.0 
N = 50  
K = 99 
r = 0.06 
vol_range = np.linspace(0.1, 0.99, 100)
N_range = [20, 50, 200]
#Plot the delta values for different volatilities using Black-Scholes and Binomial tree models(N = 20,50,200) in label
plt.figure(figsize=(20, 5))
for i, N in enumerate(N_range):
    bs_deltas = []
    bt_deltas = []
    for vol in vol_range:
        bs_delta = black_scholes_delta(S, K, T, r, vol)
        bs_deltas.append(bs_delta)
        bt_delta = binomial_tree_delta(S, K, T, r, vol, N)
        bt_deltas.append(bt_delta)
    plt.subplot(1, 3, i+1)
    plt.plot(vol_range, bs_deltas, label='Black-Scholes Delta')
    plt.plot(vol_range, bt_deltas, label=f'Binomial Tree Delta (N = {N})', linestyle='--')
    plt.title(f'European Call Option Delta for N = {N}', fontsize = 16)
    plt.xlabel('Volatility', fontsize=16)
    plt.ylabel('Delta', fontsize=16)
    plt.legend(fontsize = 12)
    plt.grid(True)
plt.show()


In [ ]:
sigma = 0.2  
S = 100 
T = 1.0 
N = 50  
K = 99 
r = 0.06 
vol_range = np.linspace(0.1, 0.99, 100)
N_range = [20, 50, 200]
#Plot the delta values' difference for different volatilities using Black-Scholes and Binomial tree models(N = 20,50,200) in label
plt.figure(figsize=(20, 5))
for i, N in enumerate(N_range):
    bs_deltas = []
    bt_deltas = []
    for vol in vol_range:
        bs_delta = black_scholes_delta(S, K, T, r, vol)
        bs_deltas.append(bs_delta)
        bt_delta = binomial_tree_delta(S, K, T, r, vol, N)
        bt_deltas.append(bt_delta)
    deltas_difference = np.array(bt_deltas) - np.array(bs_deltas)
    plt.subplot(1, 3, i+1)
    plt.plot(vol_range, deltas_difference, label='Difference in Delta')
    plt.title(f'European Call Option Delta for N = {N}', fontsize = 16)
    plt.xlabel('Volatility', fontsize=16)
    plt.ylabel('Delta', fontsize=16)
    plt.legend(fontsize = 12)
    plt.ylim(-0.001,0.006)
    plt.grid(True)
plt.show()

In [ ]:
sigma = 0.2  
S = 100 
T = 1.0 
N = 50  
K = 99 
r = 0.06 
vol_range = np.linspace(0.1, 0.99, 100)
N_range = [20, 50, 200]
#Plot the delta values' ralative difference for different volatilities using Black-Scholes and Binomial tree models(N = 20,50,200) in label
plt.figure(figsize=(20, 5))
for i, N in enumerate(N_range):
    bs_deltas = []
    bt_deltas = []
    for vol in vol_range:
        bs_delta = black_scholes_delta(S, K, T, r, vol)
        bs_deltas.append(bs_delta)
        bt_delta = binomial_tree_delta(S, K, T, r, vol, N)
        bt_deltas.append(bt_delta)
    deltas_relative_difference = (np.array(bt_deltas) - np.array(bs_deltas))/np.array(bs_deltas) * 100
    plt.subplot(1, 3, i+1)
    plt.plot(vol_range, deltas_relative_difference, label='Relative Difference in Delta', linestyle='--')
    plt.title(f'European Call Option Delta for N = {N}', fontsize = 16)
    plt.xlabel('Volatility', fontsize=16)
    plt.ylabel('Delta(%)', fontsize=16)
    # use % for y-axis
    plt.ylim(-0.15,1)
    plt.legend(fontsize = 12)
    plt.grid(True)
plt.show()

#### 5. American Call Option for Binomial Tree Model

In [ ]:
def value_american_option(tree, T, r, K, vol, is_call=True):
    '''
    Calculate the price of an American option using a binomial tree.
    Parameters:
        tree: Binomial tree
        T: Time to expiration
        r: Risk-free interest rate
        K: Strike price
        vol: Volatility
        is_call: Boolean value to determine if the option is a call option
    Returns:
        Option value at the root of the tree
    '''
    N = tree.shape[0] - 1  # Number of steps
    dt = T / N
    u = np.exp(vol * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)  # Risk-neutral probability

    # Initialize the option value at expiration
    for c in range(N + 1):
        if is_call:
            tree[N, c] = max(0, tree[N, c] - K)  # Call option payoff
        else:
            tree[N, c] = max(0, K - tree[N, c])  # Put option payoff

    # Iterate backwards through the tree to get the option value at the root
    for i in range(N - 1, -1, -1):
        for j in range(i + 1):
            # Calculate the expected value of the option
            hold_value = np.exp(-r * dt) * (p * tree[i + 1, j + 1] + (1 - p) * tree[i + 1, j])
            if is_call:
                exercise_value = max(0, tree[i, j] - K)  # Call option payoff
            else:
                exercise_value = max(0, K - tree[i, j])  # Put option payoff
            # For American options, take the max of holding and exercising
            tree[i, j] = max(hold_value, exercise_value)
            
    return tree[0, 0]

# Given parameters for the American option
is_call_option = True  # Set to False for put option

# Calculate the American option price for different volatilities
american_call_prices = []
american_put_prices = []
for sigma in vol_range:
    tree = build_tree(S, sigma, T, N)
    american_call_price = value_american_option(tree, T, r, K, sigma, is_call=True)
    american_call_prices.append(american_call_price)
    american_put_price = value_american_option(tree, T, r, K, sigma, is_call=False)
    american_put_prices.append(american_put_price)

# Plotting American call and put option prices for comparison
plt.figure(figsize=(7, 6))
plt.plot(vol_range, american_call_prices, label='American Call Prices', marker='o')
plt.plot(vol_range, american_put_prices, label='American Put Prices', marker='x')
plt.title('American Option Prices for Different Volatilities', fontsize=16)
plt.xlabel('Volatility', fontsize=16)
plt.ylabel('Option Price', fontsize=16)
plt.legend(fontsize=16)
plt.grid(True)
plt.show()

# Output the first few prices for review
american_call_prices[:5], american_put_prices[:5]
